In [ ]:
!pip install numpy matplotlib PyQt5 

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import os
from PyQt5.QtWidgets import (QApplication, QMainWindow, QWidget, QVBoxLayout, QHBoxLayout, 
                            QLabel, QPushButton, QFileDialog, QGroupBox, QDoubleSpinBox,
                            QSpinBox, QTabWidget, QCheckBox, QMessageBox)
from PyQt5.QtCore import Qt
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.backends.backend_qt5agg import NavigationToolbar2QT as NavigationToolbar

In [ ]:
class MeshViewer(FigureCanvas):
    def __init__(self, parent=None, width=5, height=5, dpi=100):
        self.fig = plt.figure(figsize=(width, height), dpi=dpi)
        self.ax = self.fig.add_subplot(111, projection='3d')
        super().__init__(self.fig)
        self.setParent(parent)
        self.vertices = None
        self.faces = None
    
    def plot_mesh(self, vertices, faces):
        self.vertices = vertices
        self.faces = faces
        
        self.ax.clear()
        if vertices is not None and faces is not None:
            mesh = Poly3DCollection([vertices[face] for face in faces], 
                                   alpha=0.7, edgecolor='k')
            self.ax.add_collection3d(mesh)
            self.ax.scatter(vertices[:, 0], vertices[:, 1], vertices[:, 2], 
                           s=5, color='r')
            scale = vertices.flatten()
            self.ax.auto_scale_xyz(scale, scale, scale)
        self.draw()

class MeshProcessorApp(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("3D Mesh Processing Tool")
        self.setGeometry(100, 100, 1200, 800)
        
        # Main variables
        self.vertices = None
        self.faces = None
        self.original_vertices = None
        self.file_path = ""
        
        # Initialize UI
        self.init_ui()
        
    def init_ui(self):
        # Central widget and main layout
        central_widget = QWidget()
        self.setCentralWidget(central_widget)
        main_layout = QHBoxLayout(central_widget)
        
        # Left panel for controls
        control_panel = QWidget()
        control_layout = QVBoxLayout(control_panel)
        control_layout.setAlignment(Qt.AlignTop)
        control_panel.setMaximumWidth(350)
        
        # File operations group
        file_group = QGroupBox("File Operations")
        file_layout = QVBoxLayout(file_group)
        
        self.file_label = QLabel("No file loaded")
        self.file_label.setWordWrap(True)
        
        load_btn = QPushButton("Load Mesh File")
        load_btn.clicked.connect(self.load_file)
        
        reset_btn = QPushButton("Reset to Original")
        reset_btn.clicked.connect(self.reset_mesh)
        
        file_layout.addWidget(self.file_label)
        file_layout.addWidget(load_btn)
        file_layout.addWidget(reset_btn)
        
        # Transformations group
        transform_group = QGroupBox("Transformations")
        transform_layout = QVBoxLayout(transform_group)
        
        # Translation controls
        trans_group = QGroupBox("Translation")
        trans_layout = QHBoxLayout(trans_group)
        
        self.trans_x = QDoubleSpinBox()
        self.trans_x.setRange(-100, 100)
        self.trans_x.setSingleStep(0.1)
        self.trans_x.setPrefix("X: ")
        
        self.trans_y = QDoubleSpinBox()
        self.trans_y.setRange(-100, 100)
        self.trans_y.setSingleStep(0.1)
        self.trans_y.setPrefix("Y: ")
        
        self.trans_z = QDoubleSpinBox()
        self.trans_z.setRange(-100, 100)
        self.trans_z.setSingleStep(0.1)
        self.trans_z.setPrefix("Z: ")
        
        trans_layout.addWidget(self.trans_x)
        trans_layout.addWidget(self.trans_y)
        trans_layout.addWidget(self.trans_z)
        
        # Rotation controls
        rot_group = QGroupBox("Rotation (degrees)")
        rot_layout = QHBoxLayout(rot_group)
        
        self.rot_x = QSpinBox()
        self.rot_x.setRange(-360, 360)
        self.rot_x.setPrefix("X: ")
        
        self.rot_y = QSpinBox()
        self.rot_y.setRange(-360, 360)
        self.rot_y.setPrefix("Y: ")
        
        self.rot_z = QSpinBox()
        self.rot_z.setRange(-360, 360)
        self.rot_z.setPrefix("Z: ")
        
        rot_layout.addWidget(self.rot_x)
        rot_layout.addWidget(self.rot_y)
        rot_layout.addWidget(self.rot_z)
        
        # Scale control
        scale_group = QGroupBox("Scale")
        scale_layout = QHBoxLayout(scale_group)
        
        self.scale_factor = QDoubleSpinBox()
        self.scale_factor.setRange(0.01, 10)
        self.scale_factor.setValue(1.0)
        self.scale_factor.setSingleStep(0.1)
        self.scale_factor.setPrefix("Factor: ")
        
        scale_layout.addWidget(self.scale_factor)
        
        # Apply button
        apply_btn = QPushButton("Apply Transformations")
        apply_btn.clicked.connect(self.apply_transformations)
        
        transform_layout.addWidget(trans_group)
        transform_layout.addWidget(rot_group)
        transform_layout.addWidget(scale_group)
        transform_layout.addWidget(apply_btn)
        
        # Display options
        display_group = QGroupBox("Display Options")
        display_layout = QVBoxLayout(display_group)
        
        self.wireframe_check = QCheckBox("Show Wireframe")
        self.wireframe_check.setChecked(True)
        
        self.vertices_check = QCheckBox("Show Vertices")
        self.vertices_check.setChecked(True)
        
        display_layout.addWidget(self.wireframe_check)
        display_layout.addWidget(self.vertices_check)
        
        # Add groups to control panel
        control_layout.addWidget(file_group)
        control_layout.addWidget(transform_group)
        control_layout.addWidget(display_group)
        
        # Right panel for visualization
        vis_panel = QWidget()
        vis_layout = QVBoxLayout(vis_panel)
        
        # Create matplotlib canvas
        self.canvas = MeshViewer(self)
        toolbar = NavigationToolbar(self.canvas, self)
        
        vis_layout.addWidget(toolbar)
        vis_layout.addWidget(self.canvas)
        
        # Add panels to main layout
        main_layout.addWidget(control_panel)
        main_layout.addWidget(vis_panel)
        
        # Connect signals
        self.wireframe_check.stateChanged.connect(self.update_display)
        self.vertices_check.stateChanged.connect(self.update_display)
        
    def load_file(self):
        file_path, _ = QFileDialog.getOpenFileName(
            self, "Open Mesh File", "", "3D Mesh Files (*.off *.obj)")
        
        if file_path:
            try:
                self.file_path = file_path
                self.file_label.setText(os.path.basename(file_path))
                
                ext = os.path.splitext(file_path)[1].lower()
                if ext == '.off':
                    self.vertices, self.faces = self.read_off(file_path)
                elif ext == '.obj':
                    self.vertices, self.faces = self.read_obj(file_path)
                else:
                    raise ValueError("Unsupported file format")
                
                self.original_vertices = self.vertices.copy()
                self.canvas.plot_mesh(self.vertices, self.faces)
                
                # Reset transformation controls
                self.trans_x.setValue(0)
                self.trans_y.setValue(0)
                self.trans_z.setValue(0)
                self.rot_x.setValue(0)
                self.rot_y.setValue(0)
                self.rot_z.setValue(0)
                self.scale_factor.setValue(1.0)
                
            except Exception as e:
                QMessageBox.critical(self, "Error", f"Failed to load file:\n{str(e)}")
    
    def reset_mesh(self):
        if self.original_vertices is not None:
            self.vertices = self.original_vertices.copy()
            self.canvas.plot_mesh(self.vertices, self.faces)
            
            # Reset transformation controls
            self.trans_x.setValue(0)
            self.trans_y.setValue(0)
            self.trans_z.setValue(0)
            self.rot_x.setValue(0)
            self.rot_y.setValue(0)
            self.rot_z.setValue(0)
            self.scale_factor.setValue(1.0)
    
    def apply_transformations(self):
        if self.vertices is None:
            return
            
        # Get current transformation values
        tx = self.trans_x.value()
        ty = self.trans_y.value()
        tz = self.trans_z.value()
        
        rx = self.rot_x.value()
        ry = self.rot_y.value()
        rz = self.rot_z.value()
        
        scale = self.scale_factor.value()
        
        # Apply transformations to a copy of the original vertices
        self.vertices = self.original_vertices.copy()
        self.vertices = self.translate(self.vertices, [tx, ty, tz])
        self.vertices = self.scale_mesh(self.vertices, scale)
        self.vertices = self.rotate_x(self.vertices, rx)
        self.vertices = self.rotate_y(self.vertices, ry)
        self.vertices = self.rotate_z(self.vertices, rz)
        
        self.canvas.plot_mesh(self.vertices, self.faces)
    
    def update_display(self):
        if self.vertices is not None and self.faces is not None:
            self.canvas.plot_mesh(self.vertices, self.faces)
    
    def read_off(self, file_path):
        with open(file_path, 'r') as f:
            # Skip comments and empty lines at the beginning
            line = f.readline().strip()
            while line == '' or line.startswith('#'):
                line = f.readline().strip()
            
            # Check OFF header
            if line.upper() != 'OFF':
                raise ValueError("Not a valid OFF file")
            
            # Read next non-empty line for counts
            line = f.readline().strip()
            while line == '':
                line = f.readline().strip()
            
            # Parse vertex and face counts
            parts = line.split()
            if len(parts) < 2:
                raise ValueError("Invalid OFF file format")
            
            n_verts = int(parts[0])
            n_faces = int(parts[1])
            
            # Read vertices
            vertices = []
            for _ in range(n_verts):
                line = f.readline().strip()
                while line == '':
                    line = f.readline().strip()
                parts = list(map(float, line.split()))
                if len(parts) < 3:
                    raise ValueError("Invalid vertex data")
                vertices.append(parts[:3])  # Only take first 3 coordinates
            
            # Read faces
            faces = []
            for _ in range(n_faces):
                line = f.readline().strip()
                while line == '':
                    line = f.readline().strip()
                parts = list(map(int, line.split()))
                if len(parts) < 4:
                    raise ValueError("Invalid face data")
                if parts[0] != 3:
                    raise ValueError("Only triangular faces are supported")
                faces.append(parts[1:4])  # Skip the count and take the vertex indices
            
        return np.array(vertices), faces

    def read_obj(self, file_path):
        vertices = []
        faces = []
        
        with open(file_path, 'r') as f:
            for line in f:
                line = line.strip()
                if not line or line.startswith('#'):
                    continue
                
                parts = line.split()
                if not parts:
                    continue
                    
                if parts[0] == 'v':
                    # Vertex definition
                    if len(parts) < 4:
                        continue
                    vertices.append([float(parts[1]), float(parts[2]), float(parts[3])])
                elif parts[0] == 'f':
                    # Face definition
                    face_vertices = []
                    for part in parts[1:]:
                        # Handle formats like f v1/vt1/vn1 v2/vt2/vn2 v3/vt3/vn3
                        vertex_part = part.split('/')[0]
                        if vertex_part:
                            face_vertices.append(int(vertex_part) - 1)  # Convert to 0-based index
                    
                    # Only support triangular faces
                    if len(face_vertices) >= 3:
                        # Triangulate polygon if necessary
                        for i in range(1, len(face_vertices) - 1):
                            faces.append([face_vertices[0], face_vertices[i], face_vertices[i+1]])
        
        return np.array(vertices), faces

    def translate(self, vertices, vector):
        return vertices + np.array(vector)

    def scale_mesh(self, vertices, factor):
        return vertices * factor

    def rotate_x(self, vertices, angle_deg):
        angle = np.radians(angle_deg)
        R = np.array([[1, 0, 0],
                    [0, np.cos(angle), -np.sin(angle)],
                    [0, np.sin(angle),  np.cos(angle)]])
        return vertices.dot(R.T)

    def rotate_y(self, vertices, angle_deg):
        angle = np.radians(angle_deg)
        R = np.array([[ np.cos(angle), 0, np.sin(angle)],
                    [0,             1, 0],
                    [-np.sin(angle), 0, np.cos(angle)]])
        return vertices.dot(R.T)

    def rotate_z(self, vertices, angle_deg):
        angle = np.radians(angle_deg)
        R = np.array([[np.cos(angle), -np.sin(angle), 0],
                    [np.sin(angle),  np.cos(angle), 0],
                    [0,              0,             1]])
        return vertices.dot(R.T)

if __name__ == '__main__':
    app = QApplication(sys.argv)
    window = MeshProcessorApp()
    window.show()
    sys.exit(app.exec_())